In [ ]:
import os
import numpy as np
from ase.io import read, write
from ase.constraints import FixAtoms, Hookean
from ase.md.verlet import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.optimize import FIRE
from mace.calculators.mace import MACECalculator

structures_dir = '/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/structures'
traj_dir = '/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/results'
os.makedirs(traj_dir, exist_ok=True)

mace_model_path = '/Users/sarahmatesic/.cache/mace/20231210mace128L0_energy_epoch249model'

num_slab_atoms = 32  
bond_k = 50.0        
bond_r0 = 1.54       
temperature = 300.0
md_steps = 50
dt = 0.2

files_to_run = sorted([f for f in os.listdir(structures_dir)
                       if f.startswith('config_') and f.endswith('.xyz')])

def compute_adsorption_energy(system, slab, polymer):
    E_total = system.get_potential_energy()
    E_slab = slab.get_potential_energy()
    E_polymer = polymer.get_potential_energy()
    return E_total - (E_slab + E_polymer)

def process_config(config_file):
    path = os.path.join(structures_dir, config_file)
    system = read(path)

    # split slab and polymer
    slab = system[:num_slab_atoms].copy()
    polymer = system[num_slab_atoms:].copy()

    # freeze slab
    mask = np.zeros(len(system), dtype=bool)
    mask[:num_slab_atoms] = True
    system.set_constraint(FixAtoms(mask=mask))

    # hookean bonds 
    carbon_indices = [i for i in range(num_slab_atoms, len(system), 3)]
    bond_pairs = [(carbon_indices[i], carbon_indices[i+1]) for i in range(len(carbon_indices)-1)]
    for a1, a2 in bond_pairs:
        system.constraints.append(Hookean(a1, a2, k=bond_k, rt=bond_r0))

    calc = MACECalculator(mace_model_path, dtype='float32')
    system.calc = calc
    slab.calc = calc
    polymer.calc = calc

    # small energy minimization
    FIRE(system).run(fmax=0.05, steps=100)

    MaxwellBoltzmannDistribution(system, temperature_K=temperature)
    dyn = VelocityVerlet(system, timestep=dt)
    for step in range(md_steps):
        dyn.run(1)

    traj_path = os.path.join(traj_dir, f"{config_file.replace('.xyz','')}.traj")
    write(traj_path, system)

    final_xyz = os.path.join(traj_dir, f"{config_file.replace('.xyz','')}_final.xyz")
    write(final_xyz, system)

    E_ads = compute_adsorption_energy(system, slab, polymer)
    print(f"{config_file}: Adsorption energy = {E_ads:.4f} eV")
    return E_ads

adsorption_energies = []
batch_size = 2
batch_pause = 5  # to cool laptop

for i in range(0, len(files_to_run), batch_size):
    batch = files_to_run[i:i+batch_size]
    for cfg in batch:
        E_ads = process_config(cfg)
        adsorption_energies.append((cfg, E_ads))
    print(f"Batch complete. Waiting {batch_pause}s...")
    import time; time.sleep(batch_pause)

results_file = os.path.join(traj_dir, "adsorption_energies.txt")
with open(results_file, 'w') as f:
    for cfg, E_ads in adsorption_energies:
        f.write(f"{cfg} {E_ads:.6f}\n")



/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/md/.venv/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
      Step     Time          Energy          fmax
FIRE:    0 17:26:42     -327.884923        8.041129
FIRE:    1 17:26:43     -329.558341        5.347364
FIRE:    2 17:26:43     -331.007573        2.976269
FIRE:    3 17:26:43     -331.489840        5.819813
FIRE:    4 17:26:43     -331.794136        3.802242
FIRE:    5 17:26:43     -332.110139        2.000332
FIRE:    6 17:26:43     -332.259566        1.651493
FIRE:    7 17:26:43     -332.298107        2.772379
FIRE:    8 17:26:44     -332.319224        2.647429
FIRE:    9 17:26:44     -332.358689        2.400411
FIRE:   10 17:26:44     -332.411384        2.037654
FIRE:   11 17:26:44     -332.470649        1.569462
FIRE:   12 17:26:44     -332.529422        1.310695
FIRE:   13 17:26:44     -332.581648        1.265206
FIRE:   14 17:26:44     -332.624030        1.216421
FIRE:   15 17:26:44     -332.660947        1.155611
FIRE:   16 17:26:4

/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/md/.venv/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


      Step     Time          Energy          fmax
FIRE:    0 17:26:55     -327.648480        8.135503
FIRE:    1 17:26:55     -329.339648        5.330086
FIRE:    2 17:26:55     -330.757162        3.142701
FIRE:    3 17:26:55     -331.216199        5.968620
FIRE:    4 17:26:55     -331.527826        3.888370
FIRE:    5 17:26:55     -331.845363        2.003524
FIRE:    6 17:26:55     -331.987234        1.655917
FIRE:    7 17:26:56     -332.015968        2.801396
FIRE:    8 17:26:56     -332.037758        2.672617
FIRE:    9 17:26:56     -332.078360        2.417848
FIRE:   10 17:26:56     -332.132277        2.043326
FIRE:   11 17:26:56     -332.192375        1.559466
FIRE:   12 17:26:56     -332.251135        1.326422
FIRE:   13 17:26:56     -332.302208        1.280234
FIRE:   14 17:26:57     -332.342347        1.231325
FIRE:   15 17:26:57     -332.376119        1.170557
FIRE:   16 17:26:57     -332.409877        1.560503
FIRE:   17 17:26:57     -332.455743        1.819660
FIRE:   18 17:

/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/md/.venv/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


...

      Step     Time          Energy          fmax
FIRE:    0 17:32:18     -327.911670        8.067560
FIRE:    1 17:32:19     -329.583899        5.368197
FIRE:    2 17:32:19     -331.035978        3.013198
FIRE:    3 17:32:19     -331.516723        5.872987
FIRE:    4 17:32:19     -331.822084        3.814965
FIRE:    5 17:32:19     -332.138047        1.991249
FIRE:    6 17:32:19     -332.286378        1.651609
FIRE:    7 17:32:19     -332.324321        2.814906
FIRE:    8 17:32:19     -332.345553        2.688183
FIRE:    9 17:32:20     -332.385228        2.437575
FIRE:   10 17:32:20     -332.438187        2.069295
FIRE:   11 17:32:20     -332.497713        1.593401
FIRE:   12 17:32:20     -332.556673        1.309746
FIRE:   13 17:32:20     -332.608941        1.264327
FIRE:   14 17:32:20     -332.651185        1.215468
FIRE:   15 17:32:20     -332.687805        1.154488
FIRE:   16 17:32:20     -332.724488        1.506687
FIRE:   17 17:32:21     -332.772736        1.773569
FIRE:   18 17:

/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/md/.venv/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
      Step     Time          Energy          fmax
FIRE:    0 17:32:43     -327.798563        7.966021
FIRE:    1 17:32:43     -329.421520        5.343591
FIRE:    2 17:32:43     -330.843594        3.101457
FIRE:    3 17:32:43     -331.310327        5.812252
FIRE:    4 17:32:44     -331.611465        3.766899
FIRE:    5 17:32:44     -331.919695        1.975567
FIRE:    6 17:32:44     -332.058951        1.631066
FIRE:    7 17:32:44     -332.089546        2.820986
FIRE:    8 17:32:44     -332.110750        2.693791
FIRE:    9 17:32:44     -332.150285        2.442176
FIRE:   10 17:32:44     -332.202844        2.072185
FIRE:   11 17:32:44     -332.261528        1.593598
FIRE:   12 17:32:45     -332.319033        1.301062
FIRE:   13 17:32:45     -332.369132        1.254933
FIRE:   14 17:32:45     -332.408597        1.207238
FIRE:   15 17:32:45     -332.441932        1.147387
FIRE:   16 17:32:4

/Users/sarahmatesic/Documents/GitHub/ML4MSD---Final-Project---SarahM/hdpe_hbn_project/md/.venv/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


FIRE:    1 17:33:03     -329.637171        5.348138
FIRE:    2 17:33:03     -331.096362        2.815973
FIRE:    3 17:33:03     -331.577867        5.706002
FIRE:    4 17:33:04     -331.877512        3.740530
FIRE:    5 17:33:04     -332.191274        1.990079
FIRE:    6 17:33:04     -332.343015        1.650426
FIRE:    7 17:33:04     -332.384857        2.696983
FIRE:    8 17:33:04     -332.405383        2.578488
FIRE:    9 17:33:04     -332.443809        2.344256
FIRE:   10 17:33:04     -332.495276        2.000241
FIRE:   11 17:33:04     -332.553448        1.555859
FIRE:   12 17:33:05     -332.611572        1.308635
FIRE:   13 17:33:05     -332.663791        1.263394
FIRE:   14 17:33:05     -332.706781        1.214736
FIRE:   15 17:33:05     -332.744703        1.153971
FIRE:   16 17:33:05     -332.782560        1.394666
FIRE:   17 17:33:05     -332.831136        1.685146
FIRE:   18 17:33:05     -332.899241        1.592348
FIRE:   19 17:33:05     -332.981904        1.072638
FIRE:   20 1